In [3]:
import CNN2Head_input
import os
import tensorflow as tf
import numpy as np
import BKNetStyle2 as BKNetStyle
from const import *

In [4]:
''' PREPARE DATA '''
''' PREPARE DATA '''
smile_train, smile_test = CNN2Head_input.getSmileImage()
gender_train, gender_test = CNN2Head_input.getGenderImage()
age_train, age_test = CNN2Head_input.getAgeImage()
'''--------------------------------------------------------------------------------------------'''

Load smile image...................
Done !
Number of smile train data:  3000
---------------------------------------------------------------
Load gender image...................
Done !
Number of gender train data:  150000
---------------------------------------------------------------
Load age image...................
Done !
Number of age train data:  150000
---------------------------------------------------------------


'--------------------------------------------------------------------------------------------'

In [5]:
def one_hot(index, num_classes):
    assert index < num_classes and index >= 0
    tmp = np.zeros(num_classes, dtype=np.float32)
    tmp[index] = 1.0
    return tmp

In [ ]:
if __name__ == "__main__":
    sess = tf.InteractiveSession()
    global_step = tf.contrib.framework.get_or_create_global_step()

    x, y_, mask = BKNetStyle.Input()

    y_smile_conv, y_gender_conv, y_age_conv, phase_train, keep_prob = BKNetStyle.BKNetModel(x)
    smile_loss, gender_loss, age_loss, l2_loss, loss = BKNetStyle.selective_loss(y_smile_conv, y_gender_conv,
                                                                                 y_age_conv, y_, mask)

    train_step = BKNetStyle.train_op(loss, global_step)

    smile_mask = tf.get_collection('smile_mask')[0]
    gender_mask = tf.get_collection('gender_mask')[0]
    age_mask = tf.get_collection('age_mask')[0]
    
    y_smile = tf.get_collection('y_smile')[0]
    y_gender = tf.get_collection('y_gender')[0]
    y_age = tf.get_collection('y_age')[0]
    
    smile_correct_prediction = tf.equal(tf.argmax(y_smile_conv, 1), tf.argmax(y_smile, 1))
    gender_correct_prediction = tf.equal(tf.argmax(y_gender_conv, 1), tf.argmax(y_gender, 1))
    age_correct_prediction = tf.equal(tf.argmax(y_age_conv, 1), tf.argmax(y_age, 1))

    smile_true_pred = tf.reduce_sum(tf.cast(smile_correct_prediction, dtype=tf.float32) * smile_mask)
    gender_true_pred = tf.reduce_sum(tf.cast(gender_correct_prediction, dtype=tf.float32) * gender_mask)
    age_true_pred = tf.reduce_sum(tf.cast(age_correct_prediction, dtype=tf.float32) * age_mask)

    train_data = []
    # Mask: Smile -> 0, Gender -> 1, Age -> 2
    for i in range(len(smile_train) * 10):
        img = (smile_train[i % 3000][0] - 128) / 255.0
        label = smile_train[i % 3000][1]
        train_data.append((img, one_hot(label, 4), 0.0))
    for i in range(len(gender_train)):
        img = (gender_train[i][0] - 128) / 255.0
        label = (int)(gender_train[i][1])
        train_data.append((img, one_hot(label, 4), 1.0))
    for i in range(len(age_train)):
        img = (age_train[i][0] - 128) / 255.0
        label = (int)(age_train[i][1])
        train_data.append((img, one_hot(label, 4), 2.0))

    saver = tf.train.Saver()

    if not os.path.isfile(SAVE_FOLDER + 'model.ckpt.index'):
        print('Create new model')
        sess.run(tf.global_variables_initializer())
        print('OK')
    else:
        print('Restoring existed model')
        saver.restore(sess, SAVE_FOLDER + 'model.ckpt')
        print('OK')

    loss_summary_placeholder = tf.placeholder(tf.float32)
    tf.summary.scalar('loss', loss_summary_placeholder)
    merge_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./summary/")

    learning_rate = tf.get_collection('learning_rate')[0]

    current_epoch = (int)(global_step.eval() / (len(train_data) // BATCH_SIZE))
    for epoch in range(current_epoch + 1, NUM_EPOCHS):
        print('Epoch:', str(epoch))
        np.random.shuffle(train_data)
        train_img = []
        train_label = []
        train_mask = []

        for i in range(len(train_data)):
            train_img.append(train_data[i][0])
            train_label.append(train_data[i][1])
            train_mask.append(train_data[i][2])

        number_batch = len(train_data) // BATCH_SIZE

        avg_ttl = []
        avg_rgl = []
        avg_smile_loss = []
        avg_gender_loss = []
        avg_age_loss = []

        smile_nb_true_pred = 0
        gender_nb_true_pred = 0
        age_nb_true_pred = 0

        smile_nb_train = 0
        gender_nb_train = 0
        age_nb_train = 0
        
        print("Learning rate: %f" % learning_rate.eval())
        for batch in range(number_batch):
            print('Training on batch {0}/{1}'.format(str(batch + 1), str(number_batch)))
            top = batch * BATCH_SIZE
            bot = min((batch + 1) * BATCH_SIZE, len(train_data))
            batch_img = np.asarray(train_img[top:bot])
            batch_label = np.asarray(train_label[top:bot])
            batch_mask = np.asarray(train_mask[top:bot])

            for i in range(BATCH_SIZE):
                if batch_mask[i] == 0.0:
                    smile_nb_train += 1
                else:
                    if batch_mask[i] == 1.0:
                        gender_nb_train += 1                        
                    else:
                        age_nb_train += 1

            batch_img = CNN2Head_input.augmentation(batch_img, 48)

            ttl, sml, gel, agl, l2l, _ = sess.run([loss, smile_loss, gender_loss, age_loss, l2_loss, train_step],
                                                  feed_dict={x: batch_img, y_: batch_label, mask: batch_mask,
                                                             phase_train: True,
                                                             keep_prob: 0.5})

            smile_nb_true_pred += sess.run(smile_true_pred, feed_dict={x: batch_img, y_: batch_label, mask: batch_mask,
                                                                       phase_train: True,
                                                                       keep_prob: 0.5})

            gender_nb_true_pred += sess.run(gender_true_pred,
                                            feed_dict={x: batch_img, y_: batch_label, mask: batch_mask,
                                                       phase_train: True,
                                                       keep_prob: 0.5})
            
            age_nb_true_pred += sess.run(age_true_pred,
                                             feed_dict={x: batch_img, y_: batch_label, mask: batch_mask,
                                                        phase_train: True,
                                                        keep_prob: 0.5})

            '''--------------------------------------- DEBUG -----------------------------------------------------'''
            '''
            sm_mask, em_mask, ge_mask = sess.run([smile_mask, gender_mask, age_mask],
                                                 feed_dict={x: batch_img, y_: batch_label, mask: batch_mask,
                                                            phase_train: True,
                                                            keep_prob: 0.5})
            print('Smile mask: ', sm_mask)
            print('Gender mask', ge_mask)
            print('Age mask', ag_mask)
            print('Batch mask', batch_mask)

            y_true_sm, y_true_ge, y_true_ag = sess.run([y_smile, y_gender, y_age],
                                                       feed_dict={x: batch_img, y_: batch_label, mask: batch_mask,
                                                                  phase_train: True,
                                                                  keep_prob: 0.5})
            print('Smile label', y_true_sm)
            print('Gender label', y_true_ge)
            print('Age label', y_true_ag)
            print('Batch label', batch_label)

            y_conv_sm, y_conv_ge, y_conv_ag = sess.run([y_smile_conv, y_gender_conv, y_age_conv],
                                                       feed_dict={x: batch_img, y_: batch_label, mask: batch_mask,
                                                                  phase_train: True,
                                                                  keep_prob: 0.5})
            print('Smile conv', y_conv_sm)
            print('Gender conv', y_conv_ge)
            print('Age conv', y_conv_ag)
            
            '''
            '''---------------------------------- END OF DEBUG ----------------------------------------------------'''

            avg_ttl.append(ttl)
            avg_smile_loss.append(sml)
            avg_gender_loss.append(gel)
            avg_age_loss.append(agl)
            
            avg_rgl.append(l2l)

        smile_train_accuracy = smile_nb_true_pred * 1.0 / smile_nb_train
        gender_train_accuracy = gender_nb_true_pred * 1.0 / gender_nb_train
        age_train_accuracy = age_nb_true_pred * 1.0 / age_nb_train

        avg_smile_loss = np.average(avg_smile_loss)
        avg_gender_loss = np.average(avg_gender_loss)
        avg_age_loss = np.average(avg_age_loss)
        
        avg_rgl = np.average(avg_rgl)
        avg_ttl = np.average(avg_ttl)

        summary = sess.run(merge_summary, feed_dict={loss_summary_placeholder: avg_ttl})
        writer.add_summary(summary, global_step=epoch)

        print('\n')

        print('Smile task train accuracy: ' + str(smile_train_accuracy * 100))
        print('Gender task train accuracy: ' + str(gender_train_accuracy * 100))
        print('Age task train accuracy: ' + str(age_train_accuracy * 100))
        print('Total loss: ' + str(avg_ttl) + '. L2-loss: ' + str(avg_rgl))
        
        print('Smile loss: ' + str(avg_smile_loss))
        print('Gender loss: ' + str(avg_gender_loss))
        print('Age loss: ' + str(avg_age_loss))

        saver.save(sess, SAVE_FOLDER + 'model.ckpt')

Instructions for updating:
Please switch to tf.train.get_or_create_global_step
Restoring existed model
INFO:tensorflow:Restoring parameters from ./save/current/model.ckpt
OK
Epoch: 5
Learning rate: 0.005987
Training on batch 1/2578
Training on batch 2/2578
Training on batch 3/2578
Training on batch 4/2578
Training on batch 5/2578
Training on batch 6/2578
Training on batch 7/2578
Training on batch 8/2578
Training on batch 9/2578
Training on batch 10/2578
Training on batch 11/2578
Training on batch 12/2578
Training on batch 13/2578
Training on batch 14/2578
Training on batch 15/2578
Training on batch 16/2578
Training on batch 17/2578
Training on batch 18/2578
Training on batch 19/2578
Training on batch 20/2578
Training on batch 21/2578
Training on batch 22/2578
Training on batch 23/2578
Training on batch 24/2578
Training on batch 25/2578
Training on batch 26/2578
Training on batch 27/2578
Training on batch 28/2578
Training on batch 29/2578
Training on batch 30/2578
Training on batch 31/2

Training on batch 294/2578
Training on batch 295/2578
Training on batch 296/2578
Training on batch 297/2578
Training on batch 298/2578
Training on batch 299/2578
Training on batch 300/2578
Training on batch 301/2578
Training on batch 302/2578
Training on batch 303/2578
Training on batch 304/2578
Training on batch 305/2578
Training on batch 306/2578
Training on batch 307/2578
Training on batch 308/2578
Training on batch 309/2578
Training on batch 310/2578
Training on batch 311/2578
Training on batch 312/2578
Training on batch 313/2578
Training on batch 314/2578
Training on batch 315/2578
Training on batch 316/2578
Training on batch 317/2578
Training on batch 318/2578
Training on batch 319/2578
Training on batch 320/2578
Training on batch 321/2578
Training on batch 322/2578
Training on batch 323/2578
Training on batch 324/2578
Training on batch 325/2578
Training on batch 326/2578
Training on batch 327/2578
Training on batch 328/2578
Training on batch 329/2578
Training on batch 330/2578
T

Training on batch 598/2578
Training on batch 599/2578
Training on batch 600/2578
Training on batch 601/2578
Training on batch 602/2578
Training on batch 603/2578
Training on batch 604/2578
Training on batch 605/2578
Training on batch 606/2578
Training on batch 607/2578
Training on batch 608/2578
Training on batch 609/2578
Training on batch 610/2578
Training on batch 611/2578
Training on batch 612/2578
Training on batch 613/2578
Training on batch 614/2578
Training on batch 615/2578
Training on batch 616/2578
Training on batch 617/2578
Training on batch 618/2578
Training on batch 619/2578
Training on batch 620/2578
Training on batch 621/2578
Training on batch 622/2578
Training on batch 623/2578
Training on batch 624/2578
Training on batch 625/2578
Training on batch 626/2578
Training on batch 627/2578
Training on batch 628/2578
Training on batch 629/2578
Training on batch 630/2578
Training on batch 631/2578
Training on batch 632/2578
Training on batch 633/2578
Training on batch 634/2578
T

Training on batch 902/2578
Training on batch 903/2578
Training on batch 904/2578
Training on batch 905/2578
Training on batch 906/2578
Training on batch 907/2578
Training on batch 908/2578
Training on batch 909/2578
Training on batch 910/2578
Training on batch 911/2578
Training on batch 912/2578
Training on batch 913/2578
Training on batch 914/2578
Training on batch 915/2578
Training on batch 916/2578
Training on batch 917/2578
Training on batch 918/2578
Training on batch 919/2578
Training on batch 920/2578
Training on batch 921/2578
Training on batch 922/2578
Training on batch 923/2578
Training on batch 924/2578
Training on batch 925/2578
Training on batch 926/2578
Training on batch 927/2578
Training on batch 928/2578
Training on batch 929/2578
Training on batch 930/2578
Training on batch 931/2578
Training on batch 932/2578
Training on batch 933/2578
Training on batch 934/2578
Training on batch 935/2578
Training on batch 936/2578
Training on batch 937/2578
Training on batch 938/2578
T

Training on batch 1199/2578
Training on batch 1200/2578
Training on batch 1201/2578
Training on batch 1202/2578
Training on batch 1203/2578
Training on batch 1204/2578
Training on batch 1205/2578
Training on batch 1206/2578
Training on batch 1207/2578
Training on batch 1208/2578
Training on batch 1209/2578
Training on batch 1210/2578
Training on batch 1211/2578
Training on batch 1212/2578
Training on batch 1213/2578
Training on batch 1214/2578
Training on batch 1215/2578
Training on batch 1216/2578
Training on batch 1217/2578
Training on batch 1218/2578
Training on batch 1219/2578
Training on batch 1220/2578
Training on batch 1221/2578
Training on batch 1222/2578
Training on batch 1223/2578
Training on batch 1224/2578
Training on batch 1225/2578
Training on batch 1226/2578
Training on batch 1227/2578
Training on batch 1228/2578
Training on batch 1229/2578
Training on batch 1230/2578
Training on batch 1231/2578
Training on batch 1232/2578
Training on batch 1233/2578
Training on batch 12

...

Training on batch 1062/2578
Training on batch 1063/2578
Training on batch 1064/2578
Training on batch 1065/2578
Training on batch 1066/2578
Training on batch 1067/2578
Training on batch 1068/2578
Training on batch 1069/2578
Training on batch 1070/2578
Training on batch 1071/2578
Training on batch 1072/2578
Training on batch 1073/2578
Training on batch 1074/2578
Training on batch 1075/2578
Training on batch 1076/2578
Training on batch 1077/2578
Training on batch 1078/2578
Training on batch 1079/2578
Training on batch 1080/2578
Training on batch 1081/2578
Training on batch 1082/2578
Training on batch 1083/2578
Training on batch 1084/2578
Training on batch 1085/2578
Training on batch 1086/2578
Training on batch 1087/2578
Training on batch 1088/2578
Training on batch 1089/2578
Training on batch 1090/2578
Training on batch 1091/2578
Training on batch 1092/2578
Training on batch 1093/2578
Training on batch 1094/2578
Training on batch 1095/2578
Training on batch 1096/2578
Training on batch 10

Training on batch 1355/2578
Training on batch 1356/2578
Training on batch 1357/2578
Training on batch 1358/2578
Training on batch 1359/2578
Training on batch 1360/2578
Training on batch 1361/2578
Training on batch 1362/2578
Training on batch 1363/2578
Training on batch 1364/2578
Training on batch 1365/2578
Training on batch 1366/2578
Training on batch 1367/2578
Training on batch 1368/2578
Training on batch 1369/2578
Training on batch 1370/2578
Training on batch 1371/2578
Training on batch 1372/2578
Training on batch 1373/2578
Training on batch 1374/2578
Training on batch 1375/2578
Training on batch 1376/2578
Training on batch 1377/2578
Training on batch 1378/2578
Training on batch 1379/2578
Training on batch 1380/2578
Training on batch 1381/2578
Training on batch 1382/2578
Training on batch 1383/2578
Training on batch 1384/2578
Training on batch 1385/2578
Training on batch 1386/2578
Training on batch 1387/2578
Training on batch 1388/2578
Training on batch 1389/2578
Training on batch 13

Training on batch 1648/2578
Training on batch 1649/2578
Training on batch 1650/2578
Training on batch 1651/2578
Training on batch 1652/2578
Training on batch 1653/2578
Training on batch 1654/2578
Training on batch 1655/2578
Training on batch 1656/2578
Training on batch 1657/2578
Training on batch 1658/2578
Training on batch 1659/2578
Training on batch 1660/2578
Training on batch 1661/2578
Training on batch 1662/2578
Training on batch 1663/2578
Training on batch 1664/2578
Training on batch 1665/2578
Training on batch 1666/2578
Training on batch 1667/2578
Training on batch 1668/2578
Training on batch 1669/2578
Training on batch 1670/2578
Training on batch 1671/2578
Training on batch 1672/2578
Training on batch 1673/2578
Training on batch 1674/2578
Training on batch 1675/2578
Training on batch 1676/2578
Training on batch 1677/2578
Training on batch 1678/2578
Training on batch 1679/2578
Training on batch 1680/2578
Training on batch 1681/2578
Training on batch 1682/2578
Training on batch 16

Training on batch 1941/2578
Training on batch 1942/2578
Training on batch 1943/2578
Training on batch 1944/2578
Training on batch 1945/2578
Training on batch 1946/2578
Training on batch 1947/2578
Training on batch 1948/2578
Training on batch 1949/2578
Training on batch 1950/2578
Training on batch 1951/2578
Training on batch 1952/2578
Training on batch 1953/2578
Training on batch 1954/2578
Training on batch 1955/2578
Training on batch 1956/2578
Training on batch 1957/2578
Training on batch 1958/2578
Training on batch 1959/2578
Training on batch 1960/2578
Training on batch 1961/2578
Training on batch 1962/2578
Training on batch 1963/2578
Training on batch 1964/2578
Training on batch 1965/2578
Training on batch 1966/2578
Training on batch 1967/2578
Training on batch 1968/2578
Training on batch 1969/2578
Training on batch 1970/2578
Training on batch 1971/2578
Training on batch 1972/2578
Training on batch 1973/2578
Training on batch 1974/2578
Training on batch 1975/2578
Training on batch 19

Training on batch 2234/2578
Training on batch 2235/2578
Training on batch 2236/2578
Training on batch 2237/2578
Training on batch 2238/2578
Training on batch 2239/2578
Training on batch 2240/2578
Training on batch 2241/2578
Training on batch 2242/2578
Training on batch 2243/2578
Training on batch 2244/2578
Training on batch 2245/2578
Training on batch 2246/2578
Training on batch 2247/2578
Training on batch 2248/2578
Training on batch 2249/2578
Training on batch 2250/2578
Training on batch 2251/2578
Training on batch 2252/2578
Training on batch 2253/2578
Training on batch 2254/2578
Training on batch 2255/2578
Training on batch 2256/2578
Training on batch 2257/2578
Training on batch 2258/2578
Training on batch 2259/2578
Training on batch 2260/2578
Training on batch 2261/2578
Training on batch 2262/2578
Training on batch 2263/2578
Training on batch 2264/2578
Training on batch 2265/2578
Training on batch 2266/2578
Training on batch 2267/2578
Training on batch 2268/2578
Training on batch 22